## Modelo de ML

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
import nltk

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Cargo los datos de las películas\n",
df_movies = pd.read_csv("..\Data\movies_dataset_ETL.csv")

In [4]:
df_movies = df_movies[['overview', 'title']]

In [5]:
df_movies.head()

,overview,title
0,0,Midnight Man
1,0,Mardock Scramble: The Third Exhaust
2,Taisto Kasurinen is a Finnish coal miner whose...,Ariel
3,"An episode in the life of Nikander, a garbage ...",Shadows in Paradise
4,0,Avalanche Sharks


In [6]:
df_movies.shape

(45349, 2)

In [7]:
df_movies = df_movies.dropna().reset_index(drop=True)

In [8]:
df_movies.shape

(44408, 2)

In [9]:
# Define el tamaño de cada subconjunto
subset_size = 5551

# Calcula la cantidad total de subconjuntos necesarios
total_subsets = (len(df_movies) - 1) // subset_size + 1

# Crea una lista para almacenar las matrices TF-IDF de cada subconjunto
tfidf_matrices = []

# Divide los datos en subconjuntos de 4425 filas y procesa cada subconjunto
for i in range(total_subsets):
    start_index = i * subset_size
    end_index = min((i + 1) * subset_size, len(df_movies))
    df_subset = df_movies[start_index:end_index].copy()
    
    # Si el subconjunto tiene al menos una fila, procede a calcular TF-IDF y similitudes
    if len(df_subset) == subset_size:
        # Crea la matriz TF-IDF para el subconjunto actual
        stopwords_custom = nltk.corpus.stopwords.words('english')  # Agrega aquí stopwords personalizados
        tfidf = TfidfVectorizer(stop_words=stopwords_custom)
        tfidf_matrix = tfidf.fit_transform(df_subset['title'] + ' ' + df_subset['overview'])
        
        # Calcula las similitudes para el subconjunto actual
        cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
        
        # Agrega la matriz de similitudes a la lista de resultados
        tfidf_matrices.append(cosine_similarities)

# Concatena las matrices de similitudes de todos los subconjuntos en una sola matriz
cosine_similarities_full = np.concatenate(tfidf_matrices)

In [10]:
def recommendation(title):
    # Verificar si el título está en el DataFrame
    if title not in df_movies['title'].values:
        return f"No se encontró ninguna película con el título '{title}'."

    # Obtener el índice de la película con el título dado
    idx = df_movies[df_movies['title'] == title].index[0]

    # Obtener las puntuaciones de similitud de todas las películas con la película dada
    sim_scores = list(enumerate(cosine_similarities_full[idx]))

    # Ordenar las películas por puntaje de similitud en orden descendente
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener los índices de las películas más similares (excluyendo la película dada)
    sim_scores = sim_scores[1:6]  # Obtener las 5 películas más similares
    movie_indices = [x[0] for x in sim_scores]

    # Obtener los títulos de las películas más similares
    recommended_movies = df_movies['title'].iloc[movie_indices].tolist()

    return recommended_movies

In [11]:
# Esta es una muestra de como funciona en vivo
movie_title = "The Mask"  # Ejemplo
result = recommendation(movie_title)
print(result)

['Son of the Mask', 'Mask', 'Black Mask', 'Der Clown - Tag der Vergeltung', 'Halloween III: Season of the Witch']


In [12]:
# Crear una lista para almacenar los resultados de recomendación en csv
all_recommendations = []

# Iterar a través de todas las películas en el DataFrame
for idx, row in df_movies.iterrows():
    movie_title = row['title']
    recommendations = recommendation(movie_title)  # Obtener las recomendaciones
    all_recommendations.append({
        'movie_title': movie_title,
        'recommended_movies': ', '.join(recommendations)
    })

# Crear un DataFrame a partir de la lista de recomendaciones
recommendations_df = pd.DataFrame(all_recommendations)



In [13]:
# Guardar el DataFrame en un archivo CSV
recommendations_df.to_csv("..\\Data\\recomendaciones_peliculas.csv", index=False)

In [15]:
# Cargar el DataFrame precalculado desde el archivo CSV
pre_recommendations = pd.read_csv("..\\Data\\recomendaciones_peliculas.csv")

def get_recommendations_for_title(title, pre_recommendations):
    recommendations = pre_recommendations[pre_recommendations['movie_title'] == title]
    if not recommendations.empty:
        recommended_titles = recommendations.iloc[0]['recommended_movies']
        return recommended_titles.split(', ')
    else:
        return ["Película no encontrada en la Base de Datos."]

In [16]:
# Ejemplo de uso
title_to_search = 'Titanic'
recommended_titles = get_recommendations_for_title(title_to_search, pre_recommendations)
print(recommended_titles)

['Titanic', 'Pieces of April', 'Ghost Ship', 'The Legend of 1900', 'Flodder']
